# Mediapipe Development

In [5]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp

In [1]:
!pip install -q mediapipe



[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [6]:
#from google.colab import files
uploaded = files.upload()

for filename in uploaded:
    content = uploaded[filename]
    with open(filename, 'wb') as f:
        f.write(content)

if len(uploaded.keys()):
    IMAGE_FILE = next(iter(uploaded))
    print('Uploaded file:', IMAGE_FILE)

NameError: name 'files' is not defined

In [7]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose

# Initialize the video capture
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, or provide the path to a video file

while True:
    # Read a frame from the video capture
    success, frame = cap.read()

    if not success:
        break

    # Display the frame
    cv2.imshow("Virtual Jewelry Try-On", frame)

    # Check for 'q' key press to quit the application
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
    while True:
        # Read a frame from the video capture
        success, frame = cap.read()

        if not success:
            break

        # Convert the frame to RGB and process it with the Pose Landmarker model
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(frame_rgb)

        if pose_results.pose_landmarks:
            # Get the ear landmark coordinates
            left_ear = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR]
            right_ear = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR]

            # Calculate the position to place the earrings
            left_ear_x = int(left_ear.x * frame.shape[1])
            left_ear_y = int(left_ear.y * frame.shape[0])
            right_ear_x = int(right_ear.x * frame.shape[1])
            right_ear_y = int(right_ear.y * frame.shape[0])

            # Load and preprocess the earring images
            left_earring = cv2.imread('data/earring.png', cv2.IMREAD_UNCHANGED)
            right_earring = cv2.imread('data/earring.png', cv2.IMREAD_UNCHANGED)

            # Overlay the earrings on the frame
            # (Add your specific overlay logic here)

        # Display the frame
        cv2.imshow("Virtual Earring Try-On", frame)

        # Check for 'q' key press to quit the application
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 